In [4]:
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
import layer
import model

# TODO: apply for regression model
class HyperparameterSelection():
    
    def __init__(self, learning_rates, epochs, batch_sizes, data, model_type = "c"):
        self.learning_rates = learning_rates
        self.epochs = epochs
        self.batch_sizes = batch_sizes
        self.data = data
        self.accuracy_dict = {}
        self.cost_dict = {}
        self.dim = [124,64]
        self.model_type = model_type
        
        if (model_type == "c"):
            self.dim_init = 13
            self.def_epoch = 2000
            self.learning_rate = 0.001
            self.activ = model.sigmoid
            self.activ_deriv = model.sigmoid_derivative
        elif (model_type == "r"):
            self.dim_init = 3072
            self.def_epoch = 200
            self.learning_rate = 0.00001
            self.activ = model.linear
            self.activ_deriv = model.linear_derivative
        
        
    def select_best(self):
        if (self.model_type == "c"):
            best_accuracy = 0
        elif (self.model_type == "r"):
            best_accuracy = -1000000000
        best_model = None
        
        for lr in self.learning_rates:
            for e in self.epochs:
                for bs in self.batch_sizes:
                    print("lr = " + str(lr) + ", e = " + str(e) + ", bs = " + str(bs))
                    layers = [
                        layer.Layer(self.dim_init,124,model.relu,model.relu_backward),
                        layer.Layer(124,64,model.relu,model.relu_backward),
                        layer.Layer(64,1,self.activ,self.activ_deriv)
                    ]
                    model1 = model.Model(layers, lr, e, bs, self.model_type)
                    accuracy_val = model1.train(self.data.X_train, self.data.y_train, self.data.X_val, self.data.y_val)
                    
                    self.accuracy_dict[f"{lr}{e}{bs}"] = model1.accuracy_history
                    self.cost_dict[f"{lr}{e}{bs}"] = model1.cost_history
                    
                    if (accuracy_val > best_accuracy):
                        best_model = model1
                        best_accuracy = accuracy_val
                        
        return best_model, best_accuracy
    
    
    def select_best_dimensions(self, dimensions):
        if (self.model_type == "c"):
            best_accuracy = 0
        elif (self.model_type == "r"):
            best_accuracy = -1000000000
        best_model = None
        best_dim = []
        
        for dim in dimensions:
            print("dim = " + str(dim))
            layers = [
                layer.Layer(self.dim_init,dim[0],model.relu,model.relu_backward),
                layer.Layer(dim[0],dim[1],model.relu,model.relu_backward),
                layer.Layer(dim[1],1,self.activ,self.activ_deriv)
            ]
            model1 = model.Model(layers, self.learning_rate, self.def_epoch, 10, self.model_type)
            accuracy_val = model1.train(self.data.X_train, self.data.y_train, self.data.X_val, self.data.y_val)
            
            self.accuracy_dict[f"{dim}"] = model1.accuracy_history
            self.cost_dict[f"{dim}"] = model1.cost_history
                
            if (accuracy_val > best_accuracy):
                best_model = model1
                best_dim = dim
                best_accuracy = accuracy_val
            
        print("best dim: " + str(best_dim))
        return best_model, best_accuracy
    
    
    def select_best_activation_function(self, activ_funcions):
        if (self.model_type == "c"):
            best_accuracy = 0
        elif (self.model_type == "r"):
            best_accuracy = -1000000000
        best_model = None
        
        for f in activ_funcions:
            print("f = " + str(f))
            layers = [
                layer.Layer(self.dim_init,124,model.relu,model.relu_backward),
                layer.Layer(124,64,model.relu,model.relu_backward),
                layer.Layer(64,1,f[0],f[1])
            ]
            model1 = model.Model(layers, self.learning_rate, self.def_epoch, 10, self.model_type)
            accuracy_val = model1.train(self.data.X_train, self.data.y_train, self.data.X_val, self.data.y_val)
            
            self.accuracy_dict[f"{f}"] = model1.accuracy_history
            self.cost_dict[f"{f}"] = model1.cost_history
                
            if (accuracy_val > best_accuracy):
                best_model = model1
                best_accuracy = accuracy_val
        
        return best_model, best_accuracy
    
    
    def plot_comparison(self):
        for e in self.epochs:
            titles = []
            plt.figure(figsize=(8, 6))
            for lr in self.learning_rates:
                for bs in self.batch_sizes:
                    print("lr = " + str(lr) + ", e = " + str(e) + ", bs = " + str(bs))
                    plt.plot(range(e), self.accuracy_dict[f"{lr}{e}{bs}"], marker='.')
                    titles.append(f"lr={lr}, bs={bs}")
#                     plt.plot(e, self.accuracy_dict[f"{lr}{e}{bs}"], marker='o', linestyle='-')
#                     plot_graph(e, self.accuracy_dict[f"{lr}{e}{bs}"], "Accuracy")
#                     plot_graph(e, self.cost_dict[f"{lr}{e}{bs}"], "Loss")
                    
            plt.legend(titles) 
            plt.title('Accuracy on Epochs')
            plt.xlabel('Epochs')
            plt.ylabel("Accuracy")
            plt.grid(True)
            plt.show()
            
    
    def print_best_output(self, best_model, best_accuracy):
        print("best model")
        print("val accuracy = " + str(best_accuracy*100))
        print("learning rate = " + str(best_model.learning_rate))
        print("epochs = " + str(best_model.epochs))
        print("batch sizes = " + str(best_model.batch_size))
        
